### *Podemos rodar às segundas-feiras já que as partidas da liga são jogadas aos domingos*

In [1]:
import pandas as pd
from functions import clube_id_e_sessao, pega_dados, conecta_ao_db

In [2]:
sessao, club_id= clube_id_e_sessao()

Login: fabin99
Password: ········
Clube ID: 79085


### Informações gerais dos jogos

In [3]:
# Conecta ao banco de dados
db= conecta_ao_db('dados_sokker.db')
curs= db.cursor()

#Pega informações únicas das partidas já adicionadas para que não sejam inseridos valores duplicados na base
matches = pd.read_sql_query("SELECT * FROM matches", db)

ids= matches['matchID'].unique()
data_esperada= matches['dateExpected'].unique()
data_iniciada= matches['dateStarted'].unique()

In [4]:
# Id do clube a serem pegas as informações
id_clube= club_id

# Pegando informações das partidas retirando aquelas que já foram pegas
url= f'https://sokker.org/xml/matches-team-{club_id}.xml'
df_matches= pega_dados(url, sessao, search= 'match')

#Retirando valores nulos (partidas ainda não jogadas) e duplicados (partidas já adicionadas na base)
df_matches.dropna(inplace= True)

query= 'matchID not in @ids and dateExpected not in @data_esperada and dateStarted not in @data_iniciada'

# Retirando colunas que não salvaremos as informações
df_matches.drop(columns= ['homeTeamName', 'awayTeamName'], 
                inplace= True)

#df_matches= pd.concat([matches, df_matches])

df_matches= df_matches.query(query)
df_matches

,matchID,homeTeamID,awayTeamID,leagueID,round,season,week,day,dateExpected,dateStarted,homeTeamScore,awayTeamScore,supporters,weather,isFinished
24,39114339,98974,79085,3,0,0,978,6,2022-04-08 19:33,2022-04-08 19:39,1,1,2074,2,1
25,39118070,79085,58316,51530,0,0,979,0,2022-04-09 21:00,2022-04-09 21:00,0,4,1521,6,1
26,38974971,79085,78473,2076,5,56,979,1,2022-04-10 21:59,2022-04-10 22:00,8,0,2744,3,1


In [5]:
if len(df_matches) > 0:
    
    # Adiciona dados à tabela
    df_matches.to_sql('matches', con=db, if_exists='append', index= False)

In [6]:
df_matches = pd.read_sql_query("SELECT * FROM matches", db)
len(df_matches)

147

### Estatísticas dos jogos

In [7]:
#Pega informações únicas das estatísticas já adicionadas para que não sejam inseridos valores duplicados na base
stats = pd.read_sql_query("SELECT * FROM matchStats", db)

ids= stats['matchID'].unique()
team= stats['teamID'].unique()
terco_defensivo= stats['timeOnHalf'].unique()
posse_de_bola= stats['timePossession'].unique()

In [8]:
#lista que guardará as estatísticas de cada partida
list_stats= list()

# Pegando estatísticas

for ID in df_matches['matchID'].unique():
    
    url= f'https://sokker.org/xml/match-{ID}.xml'
    df= pega_dados(url, sessao, search= 'teamStats')
    df['matchID']= ID
    list_stats.append(df)
    


df_stats= pd.concat([stats for stats in list_stats])

# Retirando valores nulos (partidas ainda não jogadas) e duplicados (partidas já adicionadas na base)

df_stats.dropna(inplace= True)

query= 'matchID not in @ids and teamID not in @team and timeOnHalf not in @terco_defensivo and timePossession not in @posse_de_bola'

df_stats= df_stats.query(query)

df_stats

,teamID,timeOnHalf,timePossession,offsides,shoots,fouls,yellowCards,redCards,tacticName,ratingScoring,ratingPassing,ratingDefending,matchID
0,98974,28458,35601,0,11,0,0,0,PLATINUM,5,7,7,39114339
1,79085,42268,35125,1,5,2,1,0,5-2-3f,8,7,7,39114339
0,79085,38119,34866,0,3,4,2,0,5-3-2c,7,7,8,39118070
1,58316,32076,35329,2,15,4,2,0,WingOfCAT,10,9,10,39118070
0,79085,10753,46701,0,48,3,0,0,5-3-2a,8,7,6,38974971
1,78473,60324,24376,0,2,2,1,0,433,3,4,3,38974971


In [9]:
# Adicionando informações se houver alguma nova
if len(df_stats) > 0:
    
    # Adiciona dados à tabela
    df_stats.to_sql('matchStats', con=db, if_exists='append', index= False)

In [10]:
# Conferindo
stats = pd.read_sql_query("SELECT * FROM matchStats", db)
len(stats)

294

### Estatísticas da Liga

In [11]:
#Pega informações ligas já adicionadas para que não sejam inseridos valores duplicados na base
league_stats = pd.read_sql_query("SELECT * FROM leagueStats", db)

ids= league_stats['leagueID'].unique()
team= league_stats['teamID'].unique()
rodada=  league_stats['round'].unique()

In [12]:
#Pegar o id da liga desejada entrando na liga na página do sokker
league_id= 2076

url= f'https://sokker.org/xml/league-{league_id}.xml'
df_league_stats= pega_dados(url, sessao, search= 'teams', tree_top= True)

In [13]:
df_league_stats['leagueID']= league_id
df_league_stats

,teamID,round,points,wins,draws,losses,goalsScored,goalsLost,rankTotal,leagueID
0,89046,5,15,5,0,0,47,1,15546047005012,2076
1,79853,5,15,5,0,0,24,0,15524024005006,2076
2,79085,5,12,4,0,1,19,4,12515019004011,2076
3,122540,5,9,3,0,2,21,12,9509021003009,2076
4,142636,5,9,3,0,2,11,7,9504011003004,2076
5,47955,5,7,2,1,2,6,14,7492006002005,2076
6,120765,5,6,2,0,3,7,14,6493007002010,2076
7,142663,5,6,2,0,3,3,21,6482003002003,2076
8,123911,5,3,1,0,4,6,18,3488006001007,2076
9,78473,5,3,1,0,4,2,35,3467002001008,2076


In [14]:
# Adicionando informações 
if len(df_league_stats) > 0:
    
    # Adiciona dados à tabela
    df_league_stats.to_sql('leagueStats', con=db, if_exists='append', index= False)

league_stats = pd.read_sql_query("SELECT * FROM leagueStats", db)
len(league_stats)

### Liga informação geral

In [15]:
#Pega informações das ligas já adicionadas para que não sejam inseridos valores duplicados na base
leagues = pd.read_sql_query("SELECT * FROM league", db)

ids= leagues['leagueID'].unique()
league_name= leagues['name'].unique()

In [16]:
#Pegando dados da liga
df_league= pega_dados(url, sessao, search= 'info')

#Retirando coluna que não será utilizada
df_league.drop(columns= ['userID', 'round'], inplace= True)

#Retirando valores duplicados (se as estatísticas já tiverem sido adicionadas na base)
query= 'leagueID not in @ids and name not in @league_name'
df_league= df_league.query(query)

df_league

,leagueID,name,countryID,division,season,type,isOfficial,isCup


In [17]:
# Adicionando informações se for uma liga ainda não adicionada
if len(df_league) > 0:
    
    # Adiciona dados à tabela
    df_league.to_sql('league', con=db, if_exists='append', index= False)

In [18]:
league = pd.read_sql_query("SELECT * FROM league", db)
len(league)

2

In [19]:
# Salvando alterações
db.commit()

# Fechando conexão com a base de dados
db.close()